# WiDS 2018 Datathon

Predictive Analytics for Social Impact

Competition on [Kaggle](https://www.kaggle.com/c/wids2018datathon)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from helpers import format_submission

In [2]:
seed = 37

train = pd.read_csv('data/train.csv', low_memory=False)
test = pd.read_csv('data/test.csv', low_memory=False)

print 'Train Shape: {}'.format(train.shape)
print 'Test Shape: {}'.format(test.shape)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712

Train Shape: (18255, 1235)
Test Shape: (27285, 1234)


In [3]:
label = train['is_female']
del train['is_female']

del train['train_id']
del test['test_id']

# Remove rows/columns that are missing all data
train = train.dropna(axis=0, how='all')
train = train.dropna(axis=1, how='all')

# Convert to categorical
train_str = train.applymap(str)
train_dum = pd.get_dummies(train_str)

# Split into train and validation set
X_train, X_test, y_train, y_test = train_test_split(train_dum, label, test_size=0.2, random_state=42)

In [8]:
# Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

print 'Logistic Regression'
print 'ROC AUC Score: {}'.format(score)

Logistic Regression
ROC AUC Score: 0.884670439173


In [9]:
# Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

print 'Random Forest'
print 'ROC AUC Score: {}'.format(score)

Random Forest
ROC AUC Score: 0.856606769533


In [10]:
# Multi-layer Perceptron
clf = MLPClassifier()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

print 'Multi-layer Perceptron'
print 'ROC AUC Score: {}'.format(score)

Multi-layer Perceptron
ROC AUC Score: 0.889215622446


In [8]:
# XGBoost Classifier with default parameters
clf = XGBClassifier()
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)

XGBoost Classifier
ROC AUC Score: 0.90253559105


In [6]:
# Try max_depth=10 with XGBoost Classifier
clf = XGBClassifier(max_depth=10)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)

XGBoost Classifier
ROC AUC Score: 0.911228090164


In [4]:
# Fit XGBoost Classifier, specifying eval_metric='auc'
start = time.time()
clf = XGBClassifier(max_depth=10, n_estimators=100, random_state=seed)
clf.fit(X_train, y_train, eval_metric='auc')

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

end = time.time()
run_time = float(end - start)/60

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)
print 'Run time: {:2} minutes'.format(run_time)

XGBoost Classifier
ROC AUC Score: 0.911228090164
Run time: 21.7721663992 minutes


In [4]:
# Repeat above with n_jobs=2 to compare run times
start = time.time()
clf = XGBClassifier(max_depth=10, n_estimators=100, n_jobs=2, random_state=seed)
clf.fit(X_train, y_train, eval_metric='auc')

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

end = time.time()
run_time = float(end - start)/60

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)
print 'Run time: {} minutes'.format(run_time)

XGBoost Classifier
ROC AUC Score: 0.911228090164
Run time: 25.4510277033 minutes


In [5]:
# Fit XGBoost Classifier, increase n_estimators to 200
start = time.time()

clf = XGBClassifier(max_depth=10, n_estimators=200, random_state=seed)
clf.fit(X_train, y_train, eval_metric='auc')

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

end = time.time()
run_time = float(end - start)/60

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)
print 'Run time: {} minutes'.format(run_time)

XGBoost Classifier
ROC AUC Score: 0.912035128077
Run time: 53.4536111633 minutes


In [8]:
# Fit XGBoost Classifier, increase max_depth to 20, keep n_estimators at default (100)
start = time.time()

clf = XGBClassifier(max_depth=20, n_estimators=100, random_state=seed)
clf.fit(X_train, y_train, eval_metric='auc')

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

end = time.time()
run_time = float(end - start)/60

print 'XGBoost Classifier'
print 'ROC AUC Score: {}'.format(score)
print 'Run time: {} minutes'.format(run_time)

XGBoost Classifier
ROC AUC Score: 0.909064233888
Run time: 43.1484755158 minutes


In [13]:
# Tune max_depth (1-10) for XGBoost Classifier
start = time.time()

parameters = {'max_depth': range(1, 11, 2)}
xgb = XGBClassifier()
clf = GridSearchCV(xgb, parameters, scoring='roc_auc', refit=True)
clf.fit(X_train, y_train)

preds = clf.predict(X_test)
score = roc_auc_score(y_test, preds)

end = time.time()
run_time = float(end - start)/60

print 'XGBoost Classifier Tuned with GridSearchCV'
print 'Best parameters: {}'.format(clf.best_params_)
print 'Best GridSearchCV ROC AUC score: {}'.format(clf.best_score_)
print 'ROC AUC Score: {}'.format(score)
print 'Run time: {} minutes'.format(run_time)

XGBoost Classifier Tuned with GridSearchCV
Best parameters: {'max_depth': 7}
Best GridSearchCV ROC AUC score: 0.967213341827
ROC AUC Score: 0.911375481962
Run time: 139.383742003 minutes


In [6]:
# Format test set
test_str = test.applymap(str)
total = pd.concat([train_str, test_str], ignore_index=True)
total_dummies = pd.get_dummies(total)
train_dummies = total_dummies.head(len(train))
test_dummies = total_dummies.tail(len(test))

# # Export modified train and test sets to CSV to use with Keras
# train_dummies.to_csv('data/x_train.csv', index=False)
# pd.DataFrame(label).to_csv('data/y_train.csv', index=False)
# test_dummies.to_csv('data/x_test.csv', index=False)

In [14]:
# Update submission number 
sub_number = 9

# Fit best model to full train set
start = time.time()
print 'Fitting model...\n'
clf = XGBClassifier(max_depth=7, n_estimators=100, random_state=seed)
clf.fit(train_dummies, label)

# Make predictions
print 'Making predictions...\n'
preds = clf.predict_proba(test_dummies)

end = time.time()
run_time = float(end - start)/60
print 'Run time: {:0.2f} minutes'.format(run_time)

# Format submission 
print '\nFormatting submission...\n'
df = format_submission(preds, len(test_dummies))
df.to_csv('submissions/submission{}.csv'.format(sub_number), index=False)

print 'Program complete!'

Fitting model...

Making predictions...

Run time: 23.66 minutes

Formatting submission...

Program complete!
